# Ultralytics YOLO format

## 1. Directory

```powershell
<Custom Dataset Directory>
****├── 0.jpg
├── 0.txt
└── ...
```

### Split Dataset Directory

```powershell
rico_yolo
├── test
│   ├── 0.jpg
│   ├── 0.txt
│   └── ...
├── train
│   ├── 2.jpg
│   ├── 2.txt
│   └── ...
└── val
    ├── 36.jpg
    ├── 36.txt
    └── ...
```

## 2. Annotation

```
<object-class> <x> <y> <width> <height>
```

### 예시) 2.txt

13 0.5 0.4833984375 0.7944444444444444 0.626171875
3 0.5 0.758203125 0.7944444444444444 0.0765625
21 0.7875 0.758203125 0.16111111111111112 0.065625
6 0.5 0.4451171875 0.7944444444444444 0.549609375
21 0.5 0.198046875 0.6777777777777778 0.05546875
21 0.35381944444444446 0.2568359375 0.3854166666666667 0.062109375
8 0.2013888888888889 0.358984375 0.11666666666666667 0.065625
8 0.7986111111111112 0.358984375 0.11666666666666667 0.065625

## Dataset File Format

In [16]:
yaml_text = """
path: /weven/datasets/
train: /weven/datasets/Object/train
val: /weven/datasets/rico_yolo/val
test: /weven/datasets/rico_yolo/test

names:
    0: 'typo_text',
    1: 'typo_price_num',
    2: 'typo_price_dollar',
    3: 'typo_price_W',
    4: 'typo_price_won_en',
    5: 'typo_price_won_kr',
    6: 'image_photo',
    7: 'image_colorBG',
    8: 'image_removeBG',
    9: 'icon_arrow_left',
    10: 'icon_arrow_top',
    11: 'icon_arrow_bottom',
    12: 'icon_arrow_right',
    13: 'icon_video_play',
    14: 'icon_SNS_insta',
    15: 'icon_SNS_youtube',
    16: 'btn_radius',
    17: 'btn_ellipse',
    18: 'btn_square'"""

with open('/tf/notebook/datasets/data.yaml', 'w') as file:
    file.write(yaml_text)

In [ ]:
### 작업 디렉토리

'''powershell
weven
├── datasets
│   └── Obejectblock
└── notebooks
    └── read_rico_dataset.ipynb

'''
# Import Libraries

In [2]:
# Step 3 Training - Preparing  Dataset for YOLO




from pathlib import Path
import json
import functools
import itertools
import shutil
import numpy as np
import pandas as pd
from PIL import Image




## For parallel computing


In [ ]:
import concurrent.futures
num_workers = 32  # The number of threads or processes

# mt_executor = concurrent.futures.ThreadPoolExecutor(num_workers)  
# Use ThreadPoolExecutor for CPU-bound tasks.
# mp_executor = concurrent.futures.ProcessPoolExecutor(num_workers)  
# Use ProcessPoolExecutor for I/O-bound tasks.

#  dataset path settings

In [ ]:
screenshots_path = Path('../datasets/Object/~')  # 66261 '.jpg' and '.json'
annotations_path = Path('../datasets/Object/~/')  # 66261 '.png' and 'json'

image_ids = pd.Series([int(p.stem) for p in screenshots_path.iterdir() if p.suffix == '.json'], name='image_id')
image_ids = image_ids.sort_values().reset_index(drop='index')

screenshot_paths = [screenshots_path / f'{image_id}.jpg' for image_id in image_ids]
annotation_paths = [annotations_path / f'{image_id}.png' for image_id in image_ids]
view_hierarchy_json_paths = [screenshots_path / f'{image_id}.json' for image_id in image_ids]
annotation_hierarchy_json_paths = [annotations_path / f'{image_id}.json' for image_id in image_ids]





## Read hierarchy json files


In [ ]:
def get_hierarchy_from_json(json_path):
    with open(json_path) as fp:
        hierarchy = json.load(fp)
        return hierarchy

with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    annotation_hierarchies = mp_executor.map(get_hierarchy_from_json, annotation_hierarchy_json_paths)
    annotation_hierarchies = list(annotation_hierarchies)

with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    view_hierarchies = mp_executor.map(get_hierarchy_from_json, view_hierarchy_json_paths)
    view_hierarchies = list(view_hierarchies)

# Get Image sizes (가로, 세로)


In [ ]:
def get_imsize(path):
    im = Image.open(path)
    return im.size

with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    screenshot_sizes = mp_executor.map(get_imsize, screenshot_paths)
    screenshot_sizes = list(screenshot_sizes)

with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    annotation_sizes = mp_executor.map(get_imsize, annotation_paths)
    annotation_sizes = list(annotation_sizes)

screenshot_sizes_df = pd.DataFrame(screenshot_sizes, columns=['width', 'height'])
annotation_sizes_df = pd.DataFrame(annotation_sizes, columns=['width', 'height'])

screenshot_sizes_df.insert(0, 'image_id', image_ids)
annotation_sizes_df.insert(0, 'image_id', image_ids)
screenshot_sizes_df['ratio'] = screenshot_sizes_df['width'] / screenshot_sizes_df['height']



# Get useful information from hierarchies

## A. From view_hierarchies

### 1. root size


In [ ]:
def get_root_size(hierarchy):
    return hierarchy['activity']['root']['rel-bounds']

root_sizes = map(get_root_size, view_hierarchies)

root_sizes_df = pd.DataFrame(root_sizes, columns=['x1', 'y1', 'x2', 'y2'])
root_sizes_df.insert(0, 'image_id', image_ids)
root_sizes_df['ratio'] = root_sizes_df['x2'] / root_sizes_df['y2']






### 2. root classes


In [ ]:
def get_root_classes(hierarchy):
    root_class_name = hierarchy['activity']['root']['class']
    return root_class_name

root_class_names = list(map(get_root_classes, view_hierarchies))
pd.Series(root_class_names).value_counts()





## B. From annotaion_hierarchies



In [ ]:
def get_unique_attributes(hierarchy, attribute_name):
    uniques = set()

    recursive_function = functools.partial(get_unique_attributes, attribute_name=attribute_name)
    if type(hierarchy) is list:
        uniques_list = map(recursive_function, hierarchy)
        uniques |= functools.reduce(lambda x,y: x | y, uniques_list, set())
    
    if type(hierarchy) is dict:
        if attribute_name in hierarchy:
            uniques.add(hierarchy[attribute_name])
    
        if 'children' in hierarchy:
            uniques |= recursive_function(hierarchy['children'])
    
    return uniques



### 1. componentLabel, 2. iconClass, 3.textButtonClass

In [ ]:
all_component_labels = get_unique_attributes(annotation_hierarchies, 'componentLabel')
icon_classes = get_unique_attributes(annotation_hierarchies, 'iconClass')
text_button_classes = get_unique_attributes(annotation_hierarchies, 'textButtonClass')

all_component_labels = sorted(list(all_component_labels))
icon_classes = sorted(list(icon_classes))
text_button_classes = sorted(list(text_button_classes))


# Get bounding boxes


In [ ]:
def get_component_and_bounds(hierarchy):
    bboxes = []

    if 'componentLabel' in hierarchy:
        label = hierarchy['componentLabel']
        x1, y1, x2, y2 = hierarchy['bounds']
        bboxes.append(dict(label=label, x1=x1, y1=y1, x2=x2, y2=y2))

    if 'children' in hierarchy:
        bboxes += list(itertools.chain.from_iterable(map(get_component_and_bounds, hierarchy['children'])))
    
    return bboxes

all_bboxes = list(map(get_component_and_bounds, annotation_hierarchies))


# Preprocess dataset

### Drop duplicated boxes



In [ ]:

all_bbox_dfs = list(map(lambda x: pd.DataFrame(x).drop_duplicates().reset_index(drop=True), all_bboxes))



all_bbox_df = pd.concat(all_bbox_dfs, keys=image_ids, names=['image_id', 'bbox_id'])
all_bbox_df = all_bbox_df.convert_dtypes()

### **Find landscape view (wrong matches)**


In [ ]:
landscape_screenshot_df = screenshot_sizes_df[screenshot_sizes_df['ratio'] > 1]
landscaped = all_bbox_df.index.isin(landscape_screenshot_df['image_id'], level='image_id')



### **Find overflowed boxes**


In [ ]:

annotation_sizes = {'width': 1440, 'height': 2560}
overflowed = all_bbox_df['x1'] < 0
overflowed |= all_bbox_df['y1'] < 0
overflowed |= all_bbox_df['x2'] > annotation_sizes['width']
overflowed |= all_bbox_df['y2'] > annotation_sizes['height']

### **Find boxes with 0 or negative values**

In [ ]:

negated = all_bbox_df['x1'] >= all_bbox_df['x2']
negated |= all_bbox_df['y1'] >= all_bbox_df['y2']


### **Filter out all corrupted boxes**

In [ ]:
bbox_df = all_bbox_df[~(landscaped | overflowed | negated)]

# **Convert to YOLO format**

In [ ]:
class_index = pd.Series(list(all_component_labels))
class_index = pd.Series(class_index.index.values, index=class_index)



yolo_bbox_df = pd.DataFrame({
    'label': pd.Series(class_index[bbox_df['label']].values, index=bbox_df.index),
    'x': (bbox_df['x1'] + bbox_df['x2']) / 2 / annotation_sizes['width'],
    'y': (bbox_df['y1'] + bbox_df['y2']) / 2 / annotation_sizes['height'],
    'width': (bbox_df['x2'] - bbox_df['x1']) / annotation_sizes['width'],
    'height': (bbox_df['y2'] - bbox_df['y1']) / annotation_sizes['height'],
})


# Create YOLO dataset

In [3]:
object_yolo_path = Path('tf/datasets/datasets')
object_yolo_path.mkdir(parents=True, exist_ok=True)

### Create txt file for each image


In [4]:
for image_id, image_bboxes_df in yolo_bbox_df.groupby('image_id'):
    image_bboxes_df.to_csv(object_yolo_path / f'{image_id}.txt', sep=' ', header=False, index=False)

NameError: name 'yolo_bbox_df' is not defined

### Create dataset yaml file

In [8]:
!

/tf/notebook


In [11]:
import os
import shutil
import random

# 데이터셋 폴더 경로
dataset_path = '/tf/notebook/datasets'
train_path = 'train'
test_path = 'test'
valid_path = 'valid'

# train, test, valid 폴더 생성
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)
os.makedirs(valid_path, exist_ok=True)

# 데이터셋 파일 리스트 생성
file_list = os.listdir(dataset_path)

# 파일 리스트를 무작위로 섞음
random.shuffle(file_list)

# 데이터셋을 train:test:valid = 70:15:15로 나눔
train_size = int(0.7 * len(file_list))
test_size = int(0.15 * len(file_list))



In [8]:
import os
import shutil
import random

# 데이터셋 폴더 경로
dataset_path = '4block_shoplist'
train_path = '4block_shoplist/train'
test_path = '4block_shoplist/test'
valid_path = '4block_shoplist/valid'

# train, test, valid 폴더 생성
os.makedirs(train_path + '/images', exist_ok=True)
os.makedirs(train_path + '/labels', exist_ok=True)
os.makedirs(test_path + '/images', exist_ok=True)
os.makedirs(test_path + '/labels', exist_ok=True)
os.makedirs(valid_path + '/images', exist_ok=True)
os.makedirs(valid_path + '/labels', exist_ok=True)

# 데이터셋 파일 리스트 생성
file_list = os.listdir(dataset_path)
random.shuffle(file_list)

# 데이터셋을 train:test:valid = 70:15:15로 나눔
train_size = int(0.7 * len(file_list))
test_size = int(0.15 * len(file_list))

# train 데이터 복사
for filename in file_list[:train_size]:
    if filename.endswith('.jpeg'):
        image_path = os.path.join(dataset_path, filename)
        txt_path = os.path.join(dataset_path, filename.replace('.jpeg', '.txt'))
        shutil.copy(image_path, os.path.join(train_path + '/images', filename))
        shutil.copy(txt_path, os.path.join(train_path + '/labels', filename.replace('.jpeg', '.txt')))

# test 데이터 복사
for filename in file_list[train_size:train_size+test_size]:
    if filename.endswith('.jpeg'):
        image_path = os.path.join(dataset_path, filename)
        txt_path = os.path.join(dataset_path, filename.replace('.jpeg', '.txt'))
        shutil.copy(image_path, os.path.join(test_path + '/images', filename))
        shutil.copy(txt_path, os.path.join(test_path + '/labels', filename.replace('.jpeg', '.txt')))

# validation 데이터 복사
for filename in file_list[train_size+test_size:]:
    if filename.endswith('.jpeg'):
        image_path = os.path.join(dataset_path, filename)
        txt_path = os.path.join(dataset_path, filename.replace('.jpeg', '.txt'))
        shutil.copy(image_path, os.path.join(valid_path + '/images', filename))
        shutil.copy(txt_path, os.path.join(valid_path + '/labels', filename.replace('.jpeg', '.txt')))

In [ ]:
splits = ['train', 'val', 'test']
with open('data.yaml', 'w') as f:
    for split in splits:
        split_path = (object_yolo_path.parent / split).resolve()
        f.write(f'{split}: {split_path}\n')
    
    num_labels = len(all_component_labels)

    f.write('names:\n')
    for index, label in enumerate(all_component_labels):
        f.write(f'  {index}: {label}\n')

# Copy screenshot files to object_yolopath

In [ ]:
image_ids_to_copy = yolo_bbox_df.index.get_level_values('image_id').unique()

def copy_image(image_id):
    shutil.copy(screenshots_path / f'{image_id}.jpg', rico_yolo_path / f'{image_id}.jpg')
    return True

with concurrent.futures.ProcessPoolExecutor(num_workers) as mp_executor:
    results = list(mp_executor.map(copy_image, image_ids_to_copy))

# Data Split yolo

### 데이터셋 path 설정

In [ ]:
object_yolo_path = Path('../datasets/object_yolo/all')

# Image Id get

In [ ]:
all_image_ids = map(lambda p :int(p.stem), object_yolo_path.iterdir())
all_image_ids = pd.Series(all_image_ids).unique()

# Random array create

In [5]:
num_samples = len(all_image_ids)
seed_id = 0
np.ramdom.sedd(seed_id)
random_indices = np.random.choice(num_samples, num_samples, replace=False)

NameError: name 'all_image_ids' is not defined

# Data Split

In [ ]:
splits = ['train', 'val', 'test']
split_ratio = [0.8, 0.1, 0.1]

## 랜덤 배열을 비율에 맞추어 나누기

In [ ]:
ind = np.array(split_ratio).cumsum() * num_samples
ind = np.floor(ind).astype(int) - 1

## split별 id값 얻어내기

In [ ]:
image_ids = {
    'train': all_image_ids[:ind[0]],
    'val': all_image_ids[ind[0]:ind[1]],
    'test': all_image_ids[ind[1]:],
}

## 각 split 폴더로 복사하기

In [ ]:
for split in splits:
    object_yolo_split_path = object_yolo_path.parent / split
    object_yolo_split_path.mkdir(exist_ok=True)
    for image_id in image_ids[split]:
        shutil.copy(object_yolo_path / f'{image_id}.jpg', object_yolo_split_path / f'{image_id}.jpg')
        shutil.copy(object_yolo_path / f'{image_id}.txt', object_yolo_split_path / f'{image_id}.txt')

# Docker 환경

```powershell
docker run -dit --name training_yolo --gpus all --ipc=host --mount type=volume,source=rico,destination=/weven/datasets --mount type=volume,source=runs,destination=/weven/runs ultralytics/ultralytics
```

- `-d` 컨테이너 백그라운드로 실행
- `-it` 터미널 연결
- `--gpus all` NVIDIA GPU 연결
- `--ipc=host` YOLO Docker 환경 추천 세팅
    
    > IPC mode에서 Shared memory 관련 세팅인데, 자세히 알 필요는 없어 보인다.
    > 
- `--mount type=volume,source=rico,destination=/weven/datasets` 데이터 관련 볼륨 마운트
- `--mount type=volume,source=runs,destination=/weven/runs` 학습 관련 볼륨 마운트
    - `runs`라는 이름의 볼륨이 없으면 자동으로 생성된다.
- `--name training_yolo` 컨테이너 이름
- `ultralytics/ultralytics` YOLOv8 이미지

# 작업 디렉토리

- `/weven/datasets` 는 `rico` 라는 이름의 Docker volume
    - RICO 데이터셋과 전처리 결과가 남아 있다.
- `/weven/runs` 는 `runs` 라는 이름의 Docker volume
    - 학습 세팅과 결과가 저장될 볼륨

```powershell
weven
├── datasets
│   ├── RICO
│   └── rico_yolo
│       ├── all
│       ├── test
│       ├── train
│       └── val
└── runs
    ├── configs
    │   └── ym_sgd_lr_5e-3_epochs_200.yaml
    └── train
        └── rico_yolo.yaml
```

## 학습 데이터셋 연결

- `/weven/runs/train` 디렉토리에 `rico_yolo.yaml` 파일을 미리 준비해 놓는다.

### rico_yolo.yaml

```yaml
train: /weven/datasets/rico_yolo/train
val: /weven/datasets/rico_yolo/val
test: /weven/datasets/rico_yolo/test
names:
  0: Advertisement
  1: Background Image
  2: Bottom Navigation
  3: Button Bar
  4: Card
  5: Checkbox
  6: Date Picker
  7: Drawer
  8: Icon
  9: Image
  10: Input
  11: List Item
  12: Map View
  13: Modal
  14: Multi-Tab
  15: Number Stepper
  16: On/Off Switch
  17: Pager Indicator
  18: Radio Button
  19: Slider
  20: Text
  21: Text Button
  22: Toolbar
  23: Video
  24: Web View
```

# YOLOv8 Training Configs

- 학습 세팅을 설정 파일 하나로 조절할 수 있어 매우 편하다.
- `/weven/runs/configs` 에 학습 세팅을 저장해둔다.

### ym_sgd_lr_5e-3_epochs_200.yaml

```yaml
task: detect
mode: train
model: yolov8m.pt
data: rico_yolo.yaml
epochs: 200
patience: 3
batch: 16
workers: 16
project: yolov8m
name: sgd_lr_5e-3_epochs_200
optimizer: SGD
lr0: 0.005
```

- 학습 결과 저장: `/weven/runs/train/{project}/{name}`

# Training

- 실행 위치: `/weven/runs/train`

In [17]:
yolo task=detect mode=train model=yolov8m.pt data=/tf/notebook/datasets/data.yaml epochs=100 imgsz=640 batch=16

SyntaxError: invalid syntax (3788554442.py, line 1)

In [19]:
from ultralytics import YOLO

model = YOLO('yolov8m.pt')

results = model.train(data='data.yaml', 
                      epoch=100,
                     imgsz=640,
                     batch=16,
                     name='yolov8m_test')



New https://pypi.org/project/ultralytics/8.0.138 available 😃 Update with 'pip install -U ultralytics'


SyntaxError: '[31m[1mepoch[0m' is not a valid YOLO argument. Similar arguments are i.e. ['epochs=100'].

    Arguments received: ['yolo', '-f', '/root/.local/share/jupyter/runtime/kernel-2e827445-0042-4623-9959-9c1b5985f744.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of ('detect', 'segment', 'classify', 'pose')
                MODE (required) is one of ('train', 'val', 'predict', 'export', 'track', 'benchmark')
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco128.yaml model=yolov8n.pt epochs=10 lr0=0.01

    2. Predict a YouTube video using a pretrained segmentation model at image size 320:
        yolo predict model=yolov8n-seg.

In [15]:
!yolo train data=/tf/notebook/datasets/data.yaml model=yolov8l.pt epochs=50 lr0=0.01 imgsz=320
batch=16

100%|██████████████████████████████████████| 83.7M/83.7M [00:04<00:00, 20.6MB/s]
Ultralytics YOLOv8.0.138 🚀 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8191MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/tf/notebook/datasets/data.yaml, epochs=50, patience=50, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, re

                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        200       2719      0.982      0.708      0.827      0.643

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      10/50      3.33G     0.2714     0.2105     0.8162        295        320: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        200       2719      0.986      0.708      0.805      0.608

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      11/50      3.33G     0.2574     0.2006     0.8087        261        320: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        200       2719      0.987      0.713      0.822      0.661

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      12/50      3.33G     0.2499     0.1992     0.8116        287        320: 1
          

      34/50      3.34G     0.1742     0.1418     0.7968        459        320: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        200       2719      0.989       0.69      0.846      0.719

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      35/50      3.34G     0.1722     0.1385     0.7955        367        320: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        200       2719      0.988      0.707      0.849      0.726

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      36/50      3.34G     0.1726     0.1409     0.7989        305        320: 1
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        200       2719      0.984      0.705      0.845      0.729

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      37/5

In [23]:
pip install -U ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 10.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.0.132
    Uninstalling ultralytics-8.0.132:
      Successfully uninstalled ultralytics-8.0.132
Note: you may need to restart the kernel to use updated packages.


In [21]:
!yolo detect predict model=/tf/notebook/runs/detect/train14/weights/best.pt source='/tf/notebook/202307201547200720.jpeg' conf=0.1 iou=0.2 

Ultralytics YOLOv8.0.138 🚀 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8191MiB)
Model summary (fused): 268 layers, 43621257 parameters, 0 gradients, 164.9 GFLOPs

image 1/1 /tf/notebook/202307201547200720.jpeg: 192x320 7 typo_price_won_krs, 4 image_photos, 2 btn_squares, 54.4ms
Speed: 0.8ms preprocess, 54.4ms inference, 1.7ms postprocess per image at shape (1, 3, 192, 320)
Results saved to runs/detect/predict16


In [5]:
!pwd

/tf/notebook


In [6]:
!ls

 data_preprocess2.ipynb		        study
 datasets			        test.jpg
 efficientlearning		        tf
 efficientnetv2_impurity_del_v1.ipynb   tfhub_split_datasets.ipynb
 labelingtest			        traindata_Image_augmentation.ipynb
 logo.v1i.yolov8		        weblogo.v1i.yolov8
 logo.v4i.yolov8		        weblogo.v2i.yolov8
 logs				       'weblogo.v2i.yolov8 (1)'
 messageImage_1689833556852.jpg         yolov8_header.ipynb
 runs				        yolov8m.pt
 step1				        yolov8n.pt
 step2				        연습용끄적임.ipynb
 step2_preprocessing.ipynb
